In [1]:
import numpy as np
import pandas as pd

## 데이터 로드

In [2]:
data=pd.read_csv("/content/drive/MyDrive/ChatbotData .csv")

In [3]:
data_np=data.to_numpy()

In [4]:
#Q_data는 토큰화 하여 분석할 데이터
#L_dataa는 라벨 데이터
Q_data=[]
L_data=[]
for index,data_ in enumerate(data_np):
    Q_data.append(data_[0])
    if data_[2]==0:
        L_data.append("일상")
    elif data_[2]==1:
        L_data.append("부정")
    else:
        L_data.append("긍정")

In [5]:
Q_data=np.array(Q_data)
L_data=np.array(L_data)

In [6]:
print(Q_data[:5])
print(L_data[:5])

['12시 땡!' '1지망 학교 떨어졌어' '3박4일 놀러가고 싶다' '3박4일 정도 놀러가고 싶다' 'PPL 심하네']
['일상' '일상' '일상' '일상' '일상']


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(Q_data, L_data, test_size=0.33) #훈련셋과 테스트셋으로 분리

In [8]:
print(type(y_train))

<class 'numpy.ndarray'>


## 꼬꼬마를 이용한 형태소 분석

In [9]:
!pip install konlpy #--> 본인 환경에서 konlpy가 설치 안되어있을 때 주석 해제

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 460kB 51.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
from konlpy.tag import Kkma  
kkma=Kkma()  

In [11]:
kkma_data_train=[] # 각 데이터셋 별로 형태소 분석 수행
for index, data_ in enumerate(X_train):
    kkma_data_train.append(kkma.morphs(data_))
    if index%800 ==0:
        print(kkma.morphs(data_))
kkma_data_test=[]
for index, data_ in enumerate(X_test):
    kkma_data_test.append(kkma.morphs(data_))

['꽃바구니', '가', '좋', '을까', '과일', '바구니', '까', '아', '좋', '을까']
['짝', '남', '생각', '에', '잠', '을', '못', '자', '아', '.']
['새벽', '에', '전화', '가', '오', '았', '네']
['저', '어도', '어쩌', 'ㄹ', '수', '없', '는', '찌', '질', '하', 'ㄴ', '남자', '이', 'ㄴ가', '보', 'ㅂ니다', '.']
['폰', '게임', '좀', '그만', '하', '었', '으면']
['눈물', '이', '많', '아', '지', '었', '어']
['뱃살', '나오', 'ㄴ', '거', '같', '아']
['우리', '언니', '남친', '이', '날', '좋아하', '는데', '방법', '좀', '.']
['동성애', '어떻', '게', '생각', '하', '어']
['오늘', '의', '감정', '일기', '를', '쓰', '어', '보', 'ㄹ까', '하', 'ㅂ니다']


In [12]:
kkma_data_T=sum(kkma_data_train,[])

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer().fit(kkma_data_T)
#print(tfidfv.vocabulary_)
print(len(tfidfv.vocabulary_))

3233


In [14]:
model_train=[]
for index,data_ in enumerate(kkma_data_train):
    # tmp=np.zeros((90,3257),)
    #print(data_)
    tmp=tfidfv.transform(data_).toarray()
    #print(tmp,"check")
    #print(np.array(tmp).shape)
    tmp2=[3257,]
    for index2,data_2 in enumerate(tmp):
        tmp2+=data_2
    #print(np.array(tmp2).shape)
    model_train.append(tmp2)

In [15]:
model_test=[]
for index,data_ in enumerate(kkma_data_test):
    # tmp=np.zeros((90,3257),)
    #print(data_)
    tmp=tfidfv.transform(data_).toarray()
    #print(tmp,"check"")
    #print(np.array(tmp).shape)
    tmp2=[3257,]
    for index2,data_2 in enumerate(tmp):
        tmp2+=data_2
    #print(np.array(tmp2).shape)
    model_test.append(tmp2)

In [16]:
model_train=np.array(model_train) #머신러닝 분류기 모델의 입력으로 사용하기 위하여 넘파이 배열로 변환
y_train=np.array(y_train)

In [17]:
model_test=np.array(model_test) #머신러닝 분류기 모델의 입력으로 사용하기 위하여 넘파이 배열로 변환
y_test=np.array(y_test)

In [18]:
# print(model_train.shape)
# print(y_train.shape)

## 머신러닝 분류기를 이용한 분류

In [19]:
from sklearn.svm import SVC
svm=SVC(kernel="rbf",C=1,gamma=1)
svm.fit(model_train,y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [20]:
result=svm.predict(model_test)

In [21]:
print(result[:5])
print(y_test[:5])

['일상' '일상' '일상' '일상' '일상']
['일상' '부정' '일상' '일상' '부정']


In [22]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, result))

0.6886212198872373


In [23]:
## SVM 최적화

In [25]:
gamma_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
acc=0
for index,gamma_data in enumerate(gamma_list): # enumerate를 사용한 이유는 enumerate에 익숙해지기 위하여
    # 소수점 테스트
    svm=SVC(kernel="rbf",C=1,gamma=gamma_data)
    svm.fit(model_train,y_train)
    result=svm.predict(model_test)
    score=accuracy_score(y_test, result)
    if acc < score:
        acc = score
        svm_best = svm
    # 정수 테스트
    svm=SVC(kernel="rbf",C=1,gamma=gamma_data*10)
    svm.fit(model_train,y_train)
    result=svm.predict(model_test)
    score=accuracy_score(y_test, result)
    if acc < score:
        acc = score
        svm_best = svm

In [26]:
# svm 저장을 위한 라이브러리
import pickle
import joblib

In [27]:
joblib.dump(svm_best,'./svm_best.pkl')

['./svm_best.pkl']

In [28]:
svm_from_joblib = joblib.load('./svm_best.pkl') 